In [1]:
import sys  
sys.path.insert(0, '../../')

In [2]:
import optuna
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

import logging
import sys

In [3]:
seed = 0
n_samples = 800
n_features = 3
n_centers = 2
jitter = 3 * 0.01
datasets = []

feature_grid = [2, 3, 4, 5, 10, 15, 20]

In [4]:
experiment = []
rock_results = []
other_results = []

In [5]:
def getBlobDensities(n_centers):
  return np.round([3/n_centers * x for x in range(1, n_centers + 1)], decimals=2)

In [6]:
for f in feature_grid:
    step = 0
    for s in range(seed, seed+10):
        #print(f'jitter: {j}, seed: {s}')
        dataset = make_blobs(n_samples=n_samples, centers=n_centers, n_features=f, 
                             cluster_std=getBlobDensities(n_centers), random_state=s)
        run = {}
        run['step'] = step
        run['n_samples'] = n_samples
        run['n_centers'] = n_centers
        run['jitter'] = jitter
        run['features'] = f
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])
        kmeans = KMeans(n_clusters=n_centers, random_state=seed).fit(data).labels_
        eps, min_pts = (0.2, n_features * 2)
        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = n_features * 2
        run['min_pts'] = min_pts
        spectral = SpectralClustering(n_clusters=2).fit(data).labels_
        bandwidth = estimate_bandwidth(data)

        run['bandwidth'] = bandwidth
        mean_shift = MeanShift(bandwidth=bandwidth).fit(data).labels_

        rock = ROCK(tmax=15).fit(data).labels_

        gt = dataset[1]
        rock_results.append(AMI(rock, gt))
        other_results.append(np.max([AMI(kmeans, gt), AMI(dbscan, gt), AMI(spectral, gt), AMI(mean_shift, gt)]))

        run['ROCK'] = AMI(rock, gt)
        run['K_MEANS'] = AMI(kmeans, gt)
        run['DBSCAN'] = AMI(dbscan, gt)
        run['SPECTRAL'] = AMI(spectral, gt)
        run['MEAN_SHIFT'] = AMI(mean_shift, gt)

        step += 1

        experiment.append(run)

In [7]:
pd.DataFrame(experiment).to_csv('den_blobs_analysis_dim.csv')

In [8]:
pd.DataFrame(experiment)

,step,n_samples,n_centers,jitter,features,eps,min_pts,bandwidth,ROCK,K_MEANS,DBSCAN,SPECTRAL,MEAN_SHIFT
0,0,800,2,0.03,2,6,6,1.241227,0.332319,0.329595,0.178900,0.273427,0.070430
1,1,800,2,0.03,2,6,6,0.885879,0.974765,0.902360,0.748111,0.896096,0.922105
2,2,800,2,0.03,2,6,6,1.169648,0.833091,0.693364,0.099546,0.687164,0.750993
3,3,800,2,0.03,2,6,6,1.154664,0.685742,0.647760,0.155283,0.539983,0.000000
4,4,800,2,0.03,2,6,6,1.237106,-0.000427,0.290483,0.205678,0.152410,0.040063
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,5,800,2,0.03,20,6,6,3.851808,1.000000,1.000000,0.000000,1.000000,0.938556
66,6,800,2,0.03,20,6,6,4.270751,1.000000,1.000000,0.000000,1.000000,0.965723
67,7,800,2,0.03,20,6,6,4.502288,1.000000,1.000000,0.000000,1.000000,0.987511
68,8,800,2,0.03,20,6,6,4.207498,1.000000,1.000000,0.000000,1.000000,0.943037
